# Modelo Previsão Cotação IBOVESPA
- Dados de cotação diários de 28_12_2022 até 28_12_2025
- Dados tratados por Mateus Moraes da Silva

In [0]:
from pyspark.sql        import functions as F
from pyspark.sql.window import Window
from sklearn.ensemble   import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics    import accuracy_score, classification_report
import numpy as np


In [0]:
## Carregamento da tabela base

df = spark.read.format("delta").table("previsao_ibovespa.default.dados_historicos_ibovespa_tratado")
df = df.orderBy("data")


In [0]:
# Criação das médias móveis
window = Window.orderBy("data")
df = df.withColumn("MEDIA_MOVEL_5",  F.round(F.avg("ultimo").over(window.rowsBetween(-5, -1)),3)) ###  média movel de 5 dias
df = df.withColumn("MEDIA_MOVEL_10", F.round(F.avg("ultimo").over(window.rowsBetween(-10, -1)),3)) ### média movel de 10 dias
df = df.withColumn("MEDIA_MOVEL_20", F.round(F.avg("ultimo").over(window.rowsBetween(-20, -1)),3)) ### média móvel de 20 dias

# Visualização
display(df)

In [0]:
#Distância do preço do dia às médias móveis

df = df.withColumn("DISTANCIA_MM_5",  F.round(F.col("ULTIMO") / F.col("MEDIA_MOVEL_5") - 1,3))
df = df.withColumn("DISTANCIA_MM_10", F.round(F.col("ULTIMO") / F.col("MEDIA_MOVEL_10") - 1,3))
df = df.withColumn("DISTANCIA_MM_20", F.round(F.col("ULTIMO") / F.col("MEDIA_MOVEL_20") - 1,3))
display(df)


In [0]:
#Criação do campo de fechamento do próximo dia

df = df.withColumn(
    "FECHAMENTO_D1",
    F.lead("ULTIMO", 1).over(window)
)
display(df)

In [0]:
#Retorno Percentual D+1

df = df.withColumn(
    "RETORNO_D1",
    (F.col("FECHAMENTO_D1") / F.col("ULTIMO")) - 1
)


In [0]:
#Criação do Target Binário (1 para que o próximo dia aumenta e 0 para que reduza - inclui remoção dos casos em que a variação fica entre 0.3% e -0,3%)

LIMIAR = 0.003  # 0.3%

df = df.withColumn(
    "TARGET",
    F.when(F.col("RETORNO_D1") >  LIMIAR, 1)
     .when(F.col("RETORNO_D1") < -LIMIAR, 0)
     .otherwise(None)
)

df = df.dropna(subset=["TARGET"])

In [0]:
df.groupBy("TARGET").count().show()


In [0]:
#Inclusão do Retorno Diário

w = Window.orderBy("data")

df_feat = df.withColumn(
    "RETORNO_D0",
    (F.col("ULTIMO") / F.lag("ULTIMO", 1).over(w)) - 1
)
display(df_feat)

In [0]:
#Inclusão da volatilidade de 5 e 10 dias

df_feat = df_feat.withColumn(
    "VOL_5",
    F.stddev("RETORNO_D0").over(w.rowsBetween(-5, -1))
).withColumn(
    "VOL_10",
    F.stddev("RETORNO_D0").over(w.rowsBetween(-10, -1))
)
display(df_feat)

In [0]:
#Distância da Média Móvel de 20 dias
df_feat = df_feat.withColumn(
    "DIST_MM20",
    (F.col("ULTIMO") / F.col("MEDIA_MOVEL_20")) - 1
)
display(df_feat)

In [0]:
#Remoção das linhas NULL

df = df_feat.dropna(
    subset=[
        "MEDIA_MOVEL_5",
        "MEDIA_MOVEL_10",
        "MEDIA_MOVEL_20",
        "TARGET",
        "RETORNO_D0",
        "VOL_5",
        "VOL_10",
        "DIST_MM20"
    ]
)


In [0]:
# Apontamento das features para o treino do modelo

features = [
    "MEDIA_MOVEL_5",
    "MEDIA_MOVEL_10",
    "MEDIA_MOVEL_20",
    "VARIACAO",
    "RETORNO_D0",
    "VOL_5",
    "VOL_10",
    "DIST_MM20"  
]

target = "TARGET"


In [0]:
#Conversão para pandas com foco na criação da rolling window

pdf = (
    df
    .select(["data"] + features + [target])
    .orderBy("data")
    .toPandas()
)

pdf = pdf.reset_index(drop=True)

display(pdf)

In [0]:
#Setup para a Rolling Window

TRAIN_DAYS = 180
TEST_DAYS  = 30
STEP       = 10


In [0]:
#Tratamento para a criação da base de treino e a base de teste

max_data = df.select(F.max("DATA")).collect()[0][0]
data_corte = df.select(
    F.date_sub(F.max("data"), 30).alias("DATA_CORTE")
).collect()[0][0]

train_df = df.filter(F.col("DATA") < data_corte)
test_df  = df.filter(F.col("DATA") >= data_corte)

In [0]:
#Validação da data corte, e quantidade de linhas de cada base

print("Data corte:", data_corte)
print("Treino:", train_df.count())
print("Teste:", test_df.count())


In [0]:
#Conversão para Pandas
train_pd = train_df.select(features + [target]).toPandas()
test_pd  = test_df.select(features + [target]).toPandas()

X_train = train_pd[features]
y_train = train_pd[target]

X_test  = test_pd[features]
y_test  = test_pd[target]


In [0]:
#Validação da acurácia do modelo

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train, y_train)

pred_lr = model_lr.predict(X_test)

acc_lr = accuracy_score(y_test, pred_lr)

print("=== Regressão Logística ===")
print(f"Acurácia: {acc_lr:.2%}")
print(classification_report(y_test, pred_lr))

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

model_rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=4,
    min_samples_leaf=3,
    class_weight="balanced",
    random_state=42
)

model_rf.fit(X_train, y_train)

pred_rf = model_rf.predict(X_test)
acc_rf = accuracy_score(y_test, pred_rf)

print("=== RANDOM FOREST ===")
print(f"Acurácia: {acc_rf:.2%}")
print(classification_report(y_test, pred_rf))


In [0]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

model_gb = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

model_gb.fit(X_train, y_train)

pred_gb = model_gb.predict(X_test)
acc_gb = accuracy_score(y_test, pred_gb)

print("=== GRADIENT BOOSTING ===")
print(f"Acurácia: {acc_gb:.2%}")
print(classification_report(y_test, pred_gb))
